In [0]:
from pyspark.sql.functions import to_date, first, col, round

In [0]:
df_agregado = spark.read.parquet('dbfs:/databricks-results/bronze/*/*/*')
moedas = ['USD', 'EUR', 'GBP']
df_moedas = df_agregado.filter(df_agregado.moeda.isin(moedas))
df_moedas = df_moedas.withColumn('data', to_date(df_moedas.data))

In [0]:
resultado_taxas = df_moedas.groupBy('data').pivot('moeda').agg(first('taxa')).orderBy('data', ascending=False)


In [0]:
for moeda in moedas:
    resultado_valores_reais = resultado_valores_reais.withColumn(moeda, round(1/col(moeda), 4))




In [0]:
resultado_taxas = resultado_taxas.coalesce(1)
resultado_valores_reais = resultado_valores_reais.coalesce(1)

In [0]:
resultado_taxas.write.mode('overwrite').format("csv").option("header", "true").save("dbfs:/databricks-results/silver/taxas_conversao")

resultado_valores_reais.write.mode('overwrite').format("csv").option("header", "true").save("dbfs:/databricks-results/silver/valores_reais")

In [0]:
dbutils.fs.ls("dbfs:/databricks-results/silver/")